In [1]:
import logging
from utility import add_project_root_to_path

logging.basicConfig(level=logging.ERROR)

add_project_root_to_path()

In [2]:
# Building experiments configs

from fee_algorithm.fixed_fee import FixedFee
from fee_algorithm.discrete_fee_perfect_oracle import DiscreteFeePerfectOracle
from fee_algorithm.based_on_trade_count_fee import BasedOnTradeCountFee
from fee_algorithm.adaptive_fee_based_on_block_price_move import AdaptiveBasedOnPreviousBlockPriceMoveFee

from copy import deepcopy

# TODO: move to one place with historical data experiments
fee_algos_to_consider = {
    "FX_fee": FixedFee(exchange_fee_rate=0.003), # 30 bps
    "DA_fee": BasedOnTradeCountFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "BA_fee": AdaptiveBasedOnPreviousBlockPriceMoveFee(a_to_b_exchange_fee_rate=0.003, b_to_a_exchange_fee_rate=0.003), # 30 bps
    "OB_fee": DiscreteFeePerfectOracle(fee_rate_in_arbitrage_direction=0.0045, fee_rate_in_non_arbitrage_direction=0.0015), # 45/15 bps
}

In [3]:
from experiments.experiment import SyntheticDataDescription, GBMParameters
from datetime import datetime
import numpy as np

# Provided data, see synthetic_data_example.ipynb

gbm_parameters_by_alias = {
    "volatile_market": GBMParameters(
        S0=[3406.04, 1.299e-05],
        mu=[2.6092527015913387e-07, 0.0003608078612459716],
        cov_matrix=np.array([
            [5.91363022e-07, 1.20049585e-06],
            [1.20049585e-06, 3.40028539e-05]
        ])
    ),
    "calm_market": GBMParameters(
        S0=[3179.96, 1.587e-05],
        mu=[-5.439664535383297e-06, -3.1348041846428764e-06],
        cov_matrix=np.array(
            [[1.62154349e-06, 1.31591401e-06],
            [1.31591401e-06, 1.92476086e-06]]
        )
    ),
    "bull_market": GBMParameters(
        S0=[2518.75, 1.751e-05],
        mu=[8.82932404942772e-06, 1.0120694501213374e-05],
        cov_matrix=np.array(
            [[1.04648385e-06, 1.37680192e-06],
            [1.37680192e-06, 4.61054708e-06]]
        )
    ),
    "bear_market": GBMParameters(
        S0=[3539.93, 2.858e-05],
        mu=[-3.5843786641630628e-06, -4.891135343153541e-06],
        cov_matrix=np.array(
            [[9.69680101e-07, 1.32425664e-06],
            [1.32425664e-06, 3.19313094e-06]]
        )
    )
}

# Time range is used only to estimate number of "blocks" in simulation, so all time ranges are exactly 24 hours
time_range_by_alias = {
    "volatile_market": (datetime(2024, 3, 1, 12, 0, 0), datetime(2024, 3, 2, 12, 0, 0)),
    "calm_market": (datetime(2024, 8, 1, 12, 0, 0), datetime(2024, 8, 2, 12, 0, 0)),
    "bull_market": (datetime(2024, 11, 1, 12, 0, 0), datetime(2024, 11, 2, 12, 0, 0)),
    "bear_market": (datetime(2024, 4, 1, 12, 0, 0), datetime(2024, 4, 2, 12, 0, 0)),
}

synthetic_data_description_by_alias = {
    alias: SyntheticDataDescription(
        gbm_parameters=gbm_parameters_by_alias[alias],
        start_time=time_range_by_alias[alias][0],
        end_time=time_range_by_alias[alias][1],
    ) for alias in gbm_parameters_by_alias.keys()
}

random_seeds = [i for i in range(1000)]

In [4]:
from experiments.experiment import Experiment
from experiments.configs import DEFAULT_UNINFORMED_USERS_CONFIG
from user.informed_user import InformedUser

# build experiments config

config_by_fee_algo = {}

for fee_algo_name, fee_algo in fee_algos_to_consider.items():
    for data_alias, synthetic_data_description in synthetic_data_description_by_alias.items():
        config_by_fee_algo[f"{data_alias}|{fee_algo_name}"] = Experiment(
            data=synthetic_data_description, 
            fee_algorithm=deepcopy(fee_algo),
            informed_user=InformedUser(),
            uninformed_users=deepcopy(DEFAULT_UNINFORMED_USERS_CONFIG),
        )

In [5]:
from experiments.run_experiment import run_seeded_experiment

In [6]:
from tqdm import tqdm

results_by_fee_algo = {}

for fee_algo_name, experiment in tqdm(config_by_fee_algo.items()):
    results_by_fee_algo[fee_algo_name] = run_seeded_experiment(experiment, random_seeds=random_seeds)

100%|██████████| 16/16 [29:15<00:00, 109.73s/it]


In [7]:
all_experiments_results = {
    f"{fee_algo_name}|{random_seeds[i]}": results[i]
    for fee_algo_name, results in results_by_fee_algo.items()
    for i in range(len(results))
}

In [8]:
from visualizations.compare_fee_algoritms import get_experiment_summary

df = get_experiment_summary(
    all_experiments_results
)

In [9]:
df

,experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,volatile_market|FX_fee|0,72088.04,706,-7605.94,743,-71727.10,86871.67
1,volatile_market|FX_fee|1,97402.37,759,-8678.48,708,-96058.89,2249964.03
2,volatile_market|FX_fee|2,88193.43,714,-9461.41,761,-86107.02,3621769.83
3,volatile_market|FX_fee|3,70844.16,721,-8113.68,739,-70030.48,-71726.87
4,volatile_market|FX_fee|4,80918.41,741,-9442.17,724,-78801.24,1413659.80
...,...,...,...,...,...,...,...
15995,bear_market|OB_fee|995,818.27,110,-7621.65,715,2678.38,12815.88
15996,bear_market|OB_fee|996,916.49,104,-8283.13,733,3181.63,-5080.74
15997,bear_market|OB_fee|997,827.18,99,-8110.15,739,3092.97,-3776.07
15998,bear_market|OB_fee|998,725.01,89,-7919.21,702,3239.19,-4527.39


In [10]:
# 1. Split on underscores
df['split'] = df['experiment_name'].str.split('|')

# 2. Create columns based on slices of the split parts
df['data_alias'] = df['split'].apply(lambda x: x[0])
df['fee_algo']   = df['split'].apply(lambda x: x[1])
df['seed']       = df['split'].apply(lambda x: x[2])

# 3. Drop the split column
df = df.drop(columns=['split'])

In [11]:
df

,experiment_name,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss,data_alias,fee_algo,seed
0,volatile_market|FX_fee|0,72088.04,706,-7605.94,743,-71727.10,86871.67,volatile_market,FX_fee,0
1,volatile_market|FX_fee|1,97402.37,759,-8678.48,708,-96058.89,2249964.03,volatile_market,FX_fee,1
2,volatile_market|FX_fee|2,88193.43,714,-9461.41,761,-86107.02,3621769.83,volatile_market,FX_fee,2
3,volatile_market|FX_fee|3,70844.16,721,-8113.68,739,-70030.48,-71726.87,volatile_market,FX_fee,3
4,volatile_market|FX_fee|4,80918.41,741,-9442.17,724,-78801.24,1413659.80,volatile_market,FX_fee,4
...,...,...,...,...,...,...,...,...,...,...
15995,bear_market|OB_fee|995,818.27,110,-7621.65,715,2678.38,12815.88,bear_market,OB_fee,995
15996,bear_market|OB_fee|996,916.49,104,-8283.13,733,3181.63,-5080.74,bear_market,OB_fee,996
15997,bear_market|OB_fee|997,827.18,99,-8110.15,739,3092.97,-3776.07,bear_market,OB_fee,997
15998,bear_market|OB_fee|998,725.01,89,-7919.21,702,3239.19,-4527.39,bear_market,OB_fee,998


In [12]:
import pandas as pd
from typing import Optional

def show_stats_for_data_alias(data_alias, path_to_save: Optional[str] = None):
    df_filtered = df[df['data_alias'] == data_alias]


    grouped_stats = df_filtered.drop(columns=["experiment_name", "data_alias", "seed"]).groupby('fee_algo').agg(['mean', 'std'])

    formatted_stats = grouped_stats.apply(
        lambda row: [f"{row[col, 'mean']:.2f} ± {row[col, 'std']:.2f}" for col in grouped_stats.columns.levels[0]],
        axis=1
    )

    formatted_stats_df = pd.DataFrame(formatted_stats.tolist(), columns=grouped_stats.columns.levels[0], index=grouped_stats.index)

    formatted_stats_df.reset_index(inplace=True)

    display(formatted_stats_df)

    if path_to_save:
        formatted_stats_df.to_csv(path_to_save, index=False)

In [13]:
# Volatile Market

show_stats_for_data_alias(
    "volatile_market",
    "artifacts/synthetic_data/volatile_market_stats.csv"
)

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,84742.41 ± 8409.84,730.88 ± 20.52,-8773.44 ± 587.48,719.42 ± 19.26,-83220.50 ± 8167.07,1274036.87 ± 1222603.55
1,DA_fee,85620.50 ± 8478.50,733.41 ± 20.59,-8766.96 ± 615.23,718.89 ± 19.24,-84115.03 ± 8215.81,1275720.61 ± 1223441.99
2,FX_fee,85640.18 ± 8477.36,733.52 ± 20.56,-8768.77 ± 611.06,718.93 ± 19.32,-84133.68 ± 8222.04,1275484.21 ± 1223233.44
3,OB_fee,69766.64 ± 7414.82,596.71 ± 20.93,-8804.84 ± 563.87,719.27 ± 18.86,-67541.67 ± 7193.84,1256551.09 ± 1221397.47


In [14]:
# Calm Market

show_stats_for_data_alias(
    "calm_market",
    "artifacts/synthetic_data/calm_market_stats.csv"
)

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,591.39 ± 98.79,93.08 ± 11.42,-8112.76 ± 278.64,719.30 ± 18.69,3459.49 ± 218.89,-3451.87 ± 5370.22
1,DA_fee,596.56 ± 100.97,94.99 ± 11.98,-8114.68 ± 284.43,719.26 ± 18.66,3446.90 ± 223.00,-3401.69 ± 5590.94
2,FX_fee,595.07 ± 103.25,94.84 ± 11.80,-8115.92 ± 284.46,719.38 ± 18.89,3449.77 ± 225.83,-3429.50 ± 5533.93
3,OB_fee,433.70 ± 92.90,68.95 ± 11.57,-8118.36 ± 278.53,719.51 ± 18.82,3742.36 ± 224.49,-3740.13 ± 5480.56


In [15]:
# Bull Market

show_stats_for_data_alias(
    "bull_market",
    "artifacts/synthetic_data/bull_market_stats.csv"
)

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,2908.76 ± 338.42,210.90 ± 15.85,-8155.38 ± 308.11,719.32 ± 19.33,595.51 ± 400.77,727.35 ± 20346.80
1,DA_fee,2981.84 ± 343.68,215.07 ± 16.43,-8151.66 ± 317.53,719.13 ± 19.44,498.81 ± 408.75,857.87 ± 20781.69
2,FX_fee,2976.37 ± 341.09,215.04 ± 16.43,-8150.08 ± 316.68,719.37 ± 19.56,501.65 ± 406.64,863.98 ± 20738.34
3,OB_fee,2206.80 ± 311.67,159.40 ± 16.19,-8159.60 ± 291.05,719.66 ± 18.73,1557.48 ± 389.04,-201.89 ± 20698.47


In [16]:
# Bear Market

show_stats_for_data_alias(
    "bear_market",
    "artifacts/synthetic_data/bear_market_stats.csv"
)

,fee_algo,iu_markout,iu_trade_count,uu_markout,uu_trade_count,lp_markout,impermanent_loss
0,BA_fee,1166.74 ± 165.60,134.14 ± 13.23,-8112.46 ± 286.03,719.18 ± 18.90,2679.08 ± 263.80,-2006.58 ± 10156.02
1,DA_fee,1191.89 ± 169.34,137.18 ± 13.85,-8116.74 ± 294.18,719.48 ± 18.62,2641.58 ± 282.40,-1971.87 ± 10436.52
2,FX_fee,1192.67 ± 169.81,137.09 ± 13.91,-8116.28 ± 294.45,719.57 ± 19.02,2640.31 ± 280.54,-1949.61 ± 10442.02
3,OB_fee,872.16 ± 158.09,100.03 ± 13.72,-8105.17 ± 288.14,718.86 ± 19.14,3138.54 ± 278.07,-2448.72 ± 10393.04
